# ETL Vnstock data

In [1]:
#Load library
from vnstock3 import Vnstock
import pandas as pd
import numpy as np

## Extract data from vnstock

In [2]:
#ACB
stock = Vnstock().stock(symbol='ACB', source='VCI')
acb_ic = stock.finance.income_statement(period='quarter', lang = 'en')
acb_bs = stock.finance.balance_sheet(period='quarter', lang = 'en')
acb_cf = stock.finance.cash_flow(period='quarter', lang = 'en')
acb_ratio = stock.finance.ratio(period='quarter')

2024-07-27 17:28:13,735 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


In [3]:
#BID
stock = Vnstock().stock(symbol='BID', source='VCI')
bid_ic = stock.finance.income_statement(period='quarter', lang = 'en')
bid_bs = stock.finance.balance_sheet(period='quarter', lang = 'en')
bid_cf = stock.finance.cash_flow(period='quarter', lang = 'en')
bid_ratio = stock.finance.ratio(period='quarter')

2024-07-27 17:28:23,817 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


In [4]:
#CTG
stock = Vnstock().stock(symbol='CTG', source='VCI')
ctg_ic = stock.finance.income_statement(period='quarter', lang = 'en')
ctg_bs = stock.finance.balance_sheet(period='quarter', lang = 'en')
ctg_cf = stock.finance.cash_flow(period='quarter', lang = 'en')
ctg_ratio = stock.finance.ratio(period='quarter')

2024-07-27 17:28:32,309 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


In [5]:
#EIB
stock = Vnstock().stock(symbol='EIB', source='VCI')
eib_ic = stock.finance.income_statement(period='quarter', lang = 'en')
eib_bs = stock.finance.balance_sheet(period='quarter', lang = 'en')
eib_cf = stock.finance.cash_flow(period='quarter', lang = 'en')
eib_ratio = stock.finance.ratio(period='quarter')

2024-07-27 17:28:39,657 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


In [6]:
#HDB
stock = Vnstock().stock(symbol='HDB', source='VCI')
hdb_ic = stock.finance.income_statement(period='quarter', lang = 'en')
hdb_bs = stock.finance.balance_sheet(period='quarter', lang = 'en')
hdb_cf = stock.finance.cash_flow(period='quarter', lang = 'en')
hdb_ratio = stock.finance.ratio(period='quarter')

2024-07-27 17:28:45,588 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


In [7]:
#LPB
stock = Vnstock().stock(symbol='LPB', source='VCI')
lpb_ic = stock.finance.income_statement(period='quarter', lang = 'en')
lpb_bs = stock.finance.balance_sheet(period='quarter', lang = 'en')
lpb_cf = stock.finance.cash_flow(period='quarter', lang = 'en')
lpb_ratio = stock.finance.ratio(period='quarter')

2024-07-27 17:28:52,243 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


In [8]:
#MBB
stock = Vnstock().stock(symbol='MBB', source='VCI')
mbb_ic = stock.finance.income_statement(period='quarter', lang = 'en')
mbb_bs = stock.finance.balance_sheet(period='quarter', lang = 'en')
mbb_cf = stock.finance.cash_flow(period='quarter', lang = 'en')
mbb_ratio = stock.finance.ratio(period='quarter')

2024-07-27 17:28:58,452 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


In [9]:
#MSB
stock = Vnstock().stock(symbol='MSB', source='VCI')
msb_ic = stock.finance.income_statement(period='quarter', lang = 'en')
msb_bs = stock.finance.balance_sheet(period='quarter', lang = 'en')
msb_cf = stock.finance.cash_flow(period='quarter', lang = 'en')
msb_ratio = stock.finance.ratio(period='quarter')

2024-07-27 17:29:05,307 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


In [10]:
#OCB
stock = Vnstock().stock(symbol='OCB', source='VCI')
ocb_ic = stock.finance.income_statement(period='quarter', lang = 'en')
ocb_bs = stock.finance.balance_sheet(period='quarter', lang = 'en')
ocb_cf = stock.finance.cash_flow(period='quarter', lang = 'en')
ocb_ratio = stock.finance.ratio(period='quarter')

2024-07-27 17:29:12,427 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


In [11]:
#STB
stock = Vnstock().stock(symbol='STB', source='VCI')
stb_ic = stock.finance.income_statement(period='quarter', lang = 'en')
stb_bs = stock.finance.balance_sheet(period='quarter', lang = 'en')
stb_cf = stock.finance.cash_flow(period='quarter', lang = 'en')
stb_ratio = stock.finance.ratio(period='quarter')

2024-07-27 17:29:18,936 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


In [12]:
#TCB
stock = Vnstock().stock(symbol='TCB', source='VCI')
tcb_ic = stock.finance.income_statement(period='quarter', lang = 'en')
tcb_bs = stock.finance.balance_sheet(period='quarter', lang = 'en')
tcb_cf = stock.finance.cash_flow(period='quarter', lang = 'en')
tcb_ratio = stock.finance.ratio(period='quarter')

2024-07-27 17:29:24,330 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


In [13]:
#TPB
stock = Vnstock().stock(symbol='TPB', source='VCI')
tpb_ic = stock.finance.income_statement(period='quarter', lang = 'en')
tpb_bs = stock.finance.balance_sheet(period='quarter', lang = 'en')
tpb_cf = stock.finance.cash_flow(period='quarter', lang = 'en')
tpb_ratio = stock.finance.ratio(period='quarter')

2024-07-27 17:29:29,513 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


In [14]:
#VCB
stock = Vnstock().stock(symbol='VCB', source='VCI')
vcb_ic = stock.finance.income_statement(period='quarter', lang = 'en')
vcb_bs = stock.finance.balance_sheet(period='quarter', lang = 'en')
vcb_cf = stock.finance.cash_flow(period='quarter', lang = 'en')
vcb_ratio = stock.finance.ratio(period='quarter')

2024-07-27 17:29:35,154 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


In [15]:
#VIB
stock = Vnstock().stock(symbol='VIB', source='VCI')
vib_ic = stock.finance.income_statement(period='quarter', lang = 'en')
vib_bs = stock.finance.balance_sheet(period='quarter', lang = 'en')
vib_cf = stock.finance.cash_flow(period='quarter', lang = 'en')
vib_ratio = stock.finance.ratio(period='quarter')

2024-07-27 17:29:40,433 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


In [16]:
#VPB
stock = Vnstock().stock(symbol='VPB', source='VCI')
vpb_ic = stock.finance.income_statement(period='quarter', lang = 'en')
vpb_bs = stock.finance.balance_sheet(period='quarter', lang = 'en')
vpb_cf = stock.finance.cash_flow(period='quarter', lang = 'en')
vpb_ratio = stock.finance.ratio(period='quarter')

2024-07-27 17:29:45,885 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


# Transform data

In [17]:
def create_period(year, quarter): # Function to create the period column
    if quarter == 1:
        return f"{year}0331"
    elif quarter == 2:
        return f"{year}0630"
    elif quarter == 3:
        return f"{year}0930"
    elif quarter == 4:
        return f"{year}1231"

### ACB

In [18]:
#create period column
acb_bs['period'] = acb_bs.apply(lambda row: create_period(row['yearReport'], row['lengthReport']), axis=1)
acb_ratio.columns = acb_ratio.columns.get_level_values(1)
# Remove duplicated columns, keeping only the first occurrence
acb_bs = acb_bs.loc[:, ~acb_bs.columns.duplicated(keep='first')]
acb_cf = acb_cf.loc[:, ~acb_cf.columns.duplicated(keep='first')]
acb_ic = acb_ic.loc[:, ~acb_ic.columns.duplicated(keep='first')]
acb = pd.concat([acb_bs,acb_ic.iloc[:,3:],acb_cf.iloc[:,3:],acb_ratio.iloc[:,3:]], axis = 1)
acb = acb.replace(np.nan, 0.0)
acb.head()

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16352\2722087618.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  acb = acb.replace(np.nan, 0.0)


,ticker,yearReport,lengthReport,Net Inventories,Other current assets,Investment in properties,Other non-current assets,Other Reserves,Budget sources and other funds,MINORITY INTERESTS,...,Fixed Asset Turnover,Days Sales Outstanding,Days Inventory Outstanding,Days Payable Outstanding,Cash Cycle,Inventory Turnover,(ST+LT borrowings)/Equity,Debt/Equity,Fixed Asset-To-Equity,Owners' Equity/Charter Capital
0,ACB,2024,2,0.0,0.0,177005000000,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.290830,0.062380,1.674466
1,ACB,2024,1,0.0,0.0,177005000000,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.724440,0.062450,1.674423
2,ACB,2023,4,0.0,0.0,177005000000,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.130151,0.067120,1.588569
3,ACB,2023,3,0.0,0.0,177005000000,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.704559,0.066554,1.496091
4,ACB,2023,2,0.0,0.0,177005000000,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.035334,0.068278,1.407477


### BID

In [19]:
#create period column
bid_bs['period'] = bid_bs.apply(lambda row: create_period(row['yearReport'], row['lengthReport']), axis=1)
bid_ratio.columns = bid_ratio.columns.get_level_values(1)
# Remove duplicated columns, keeping only the first occurrence
bid_bs = bid_bs.loc[:, ~bid_bs.columns.duplicated(keep='first')]
bid_cf = bid_cf.loc[:, ~bid_cf.columns.duplicated(keep='first')]
bid_ic = bid_ic.loc[:, ~bid_ic.columns.duplicated(keep='first')]
bid = pd.concat([bid_bs,bid_ic.iloc[:,3:],bid_cf.iloc[:,3:],bid_ratio.iloc[:,3:]], axis = 1)
bid = bid.replace(np.nan, 0.0)
bid.head()

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16352\2874061887.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  bid = bid.replace(np.nan, 0.0)


,ticker,yearReport,lengthReport,Net Inventories,Other current assets,Investment in properties,Other non-current assets,Other Reserves,Budget sources and other funds,MINORITY INTERESTS,...,Fixed Asset Turnover,Days Sales Outstanding,Days Inventory Outstanding,Days Payable Outstanding,Cash Cycle,Inventory Turnover,(ST+LT borrowings)/Equity,Debt/Equity,Fixed Asset-To-Equity,Owners' Equity/Charter Capital
0,BID,2024,1,0.0,0.0,0,0.0,0.0,0.0,5056307000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.049681,0.084415,2.263298
1,BID,2023,4,0.0,0.0,0,0.0,0.0,0.0,4966455000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.726516,0.090275,2.156274
2,BID,2023,3,0.0,0.0,0,0.0,0.0,0.0,4840773000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.909928,0.088052,2.089101
3,BID,2023,2,0.0,0.0,0,0.0,0.0,0.0,4754650000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.613553,0.092965,2.002507
4,BID,2023,1,0.0,0.0,0,0.0,0.0,0.0,4826819000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.179736,0.094456,1.927019


### CTG

In [20]:
#create period column
ctg_bs['period'] = ctg_bs.apply(lambda row: create_period(row['yearReport'], row['lengthReport']), axis=1)
ctg_ratio.columns = ctg_ratio.columns.get_level_values(1)
# Remove duplicated columns, keeping only the first occurrence
ctg_bs = ctg_bs.loc[:, ~ctg_bs.columns.duplicated(keep='first')]
ctg_cf = ctg_cf.loc[:, ~ctg_cf.columns.duplicated(keep='first')]
ctg_ic = ctg_ic.loc[:, ~ctg_ic.columns.duplicated(keep='first')]
ctg = pd.concat([ctg_bs,ctg_ic.iloc[:,3:],ctg_cf.iloc[:,3:],ctg_ratio.iloc[:,3:]], axis = 1)
ctg = ctg.replace(np.nan, 0.0)
ctg.head()

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16352\3320170088.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ctg = ctg.replace(np.nan, 0.0)


,ticker,yearReport,lengthReport,Net Inventories,Other current assets,Investment in properties,Other non-current assets,Other Reserves,Budget sources and other funds,MINORITY INTERESTS,...,Fixed Asset Turnover,Days Sales Outstanding,Days Inventory Outstanding,Days Payable Outstanding,Cash Cycle,Inventory Turnover,(ST+LT borrowings)/Equity,Debt/Equity,Fixed Asset-To-Equity,Owners' Equity/Charter Capital
0,CTG,2024,1,0.0,0.0,0,0.0,0.0,0.0,899613000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.910365,0.075977,2.431707
1,CTG,2023,4,0.0,0.0,0,0.0,0.0,0.0,861317000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.148282,0.079372,2.375620
2,CTG,2023,3,0.0,0.0,0,0.0,0.0,0.0,837568000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.477533,0.080050,2.271980
3,CTG,2023,2,0.0,0.0,0,0.0,0.0,0.0,805548000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.755537,0.083583,2.198521
4,CTG,2023,1,0.0,0.0,0,0.0,0.0,0.0,758783000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.149094,0.089045,2.103265


### EIB

In [21]:
#create period column
eib_bs['period'] = eib_bs.apply(lambda row: create_period(row['yearReport'], row['lengthReport']), axis=1)
eib_ratio.columns = eib_ratio.columns.get_level_values(1)
# Remove duplicated columns, keeping only the first occurrence
eib_bs = eib_bs.loc[:, ~eib_bs.columns.duplicated(keep='first')]
eib_cf = eib_cf.loc[:, ~eib_cf.columns.duplicated(keep='first')]
eib_ic = eib_ic.loc[:, ~eib_ic.columns.duplicated(keep='first')]
eib = pd.concat([eib_bs,eib_ic.iloc[:,3:],eib_cf.iloc[:,3:],eib_ratio.iloc[:,3:]], axis = 1)
eib = eib.replace(np.nan, 0.0)
eib.head()

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16352\3510400318.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  eib = eib.replace(np.nan, 0.0)


,ticker,yearReport,lengthReport,Net Inventories,Other current assets,Investment in properties,Other non-current assets,Other Reserves,Budget sources and other funds,MINORITY INTERESTS,...,Fixed Asset Turnover,Days Sales Outstanding,Days Inventory Outstanding,Days Payable Outstanding,Cash Cycle,Inventory Turnover,(ST+LT borrowings)/Equity,Debt/Equity,Fixed Asset-To-Equity,Owners' Equity/Charter Capital
0,EIB,2024,1,0.0,0.0,0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.863839,0.160428,1.314746
1,EIB,2023,4,0.0,0.0,0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.973796,0.164034,1.284806
2,EIB,2023,3,0.0,0.0,0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.839336,0.166999,1.239072
3,EIB,2023,2,0.0,0.0,0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.883457,0.165922,1.226244
4,EIB,2023,1,0.0,0.0,0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.684406,0.157187,1.210736


### HDB

In [22]:
#create period column
hdb_bs['period'] = hdb_bs.apply(lambda row: create_period(row['yearReport'], row['lengthReport']), axis=1)
hdb_ratio.columns = hdb_ratio.columns.get_level_values(1)
# Remove duplicated columns, keeping only the first occurrence
hdb_bs = hdb_bs.loc[:, ~hdb_bs.columns.duplicated(keep='first')]
hdb_cf = hdb_cf.loc[:, ~hdb_cf.columns.duplicated(keep='first')]
hdb_ic = hdb_ic.loc[:, ~hdb_ic.columns.duplicated(keep='first')]
hdb = pd.concat([hdb_bs,hdb_ic.iloc[:,3:],hdb_cf.iloc[:,3:],hdb_ratio.iloc[:,3:]], axis = 1)
hdb = hdb.replace(np.nan, 0.0)
hdb.head()

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16352\3483965604.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  hdb = hdb.replace(np.nan, 0.0)


,ticker,yearReport,lengthReport,Net Inventories,Other current assets,Investment in properties,Other non-current assets,Other Reserves,Budget sources and other funds,MINORITY INTERESTS,...,Fixed Asset Turnover,Days Sales Outstanding,Days Inventory Outstanding,Days Payable Outstanding,Cash Cycle,Inventory Turnover,(ST+LT borrowings)/Equity,Debt/Equity,Fixed Asset-To-Equity,Owners' Equity/Charter Capital
0,HDB,2024,1,0.0,0.0,0,0.0,0.0,0.0,2034863000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.123713,0.030044,1.709303
1,HDB,2023,4,0.0,0.0,0,0.0,0.0,0.0,1931463000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.980834,0.030663,1.595811
2,HDB,2023,3,0.0,0.0,0,0.0,0.0,0.0,1827743000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.813048,0.030825,1.479746
3,HDB,2023,2,0.0,0.0,0,0.0,0.0,0.0,1793025000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.955897,0.032314,1.392086
4,HDB,2023,1,0.0,0.0,0,0.0,0.0,0.0,1772051000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.237509,0.031649,1.404159


### LPB

In [23]:
#create period column
lpb_bs['period'] = lpb_bs.apply(lambda row: create_period(row['yearReport'], row['lengthReport']), axis=1)
lpb_ratio.columns = lpb_ratio.columns.get_level_values(1)
# Remove duplicated columns, keeping only the first occurrence
lpb_bs = lpb_bs.loc[:, ~lpb_bs.columns.duplicated(keep='first')]
lpb_cf = lpb_cf.loc[:, ~lpb_cf.columns.duplicated(keep='first')]
lpb_ic = lpb_ic.loc[:, ~lpb_ic.columns.duplicated(keep='first')]
lpb = pd.concat([lpb_bs,lpb_ic.iloc[:,3:],lpb_cf.iloc[:,3:],lpb_ratio.iloc[:,3:]], axis = 1)
lpb = lpb.replace(np.nan, 0.0)
lpb.head()

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16352\3860648963.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lpb = lpb.replace(np.nan, 0.0)


,ticker,yearReport,lengthReport,Net Inventories,Other current assets,Investment in properties,Other non-current assets,Other Reserves,Budget sources and other funds,MINORITY INTERESTS,...,Fixed Asset Turnover,Days Sales Outstanding,Days Inventory Outstanding,Days Payable Outstanding,Cash Cycle,Inventory Turnover,(ST+LT borrowings)/Equity,Debt/Equity,Fixed Asset-To-Equity,Owners' Equity/Charter Capital
0,LPB,2024,2,0.0,0.0,0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.544326,0.062366,1.498962
1,LPB,2024,1,0.0,0.0,0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.252283,0.054237,1.423828
2,LPB,2023,4,0.0,0.0,0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.221933,0.057155,1.333953
3,LPB,2023,3,0.0,0.0,0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.796015,0.071874,1.035712
4,LPB,2023,2,0.0,0.0,0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.736790,0.070159,0.996893


### MBB

In [24]:
#create period column
mbb_bs['period'] = mbb_bs.apply(lambda row: create_period(row['yearReport'], row['lengthReport']), axis=1)
mbb_ratio.columns = mbb_ratio.columns.get_level_values(1)
# Remove duplicated columns, keeping only the first occurrence
mbb_bs = mbb_bs.loc[:, ~mbb_bs.columns.duplicated(keep='first')]
mbb_cf = mbb_cf.loc[:, ~mbb_cf.columns.duplicated(keep='first')]
mbb_ic = mbb_ic.loc[:, ~mbb_ic.columns.duplicated(keep='first')]
mbb = pd.concat([mbb_bs,mbb_ic.iloc[:,3:],mbb_cf.iloc[:,3:],mbb_ratio.iloc[:,3:]], axis = 1)
mbb = mbb.replace(np.nan, 0.0)
mbb.head()

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16352\955508960.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mbb = mbb.replace(np.nan, 0.0)


,ticker,yearReport,lengthReport,Net Inventories,Other current assets,Investment in properties,Other non-current assets,Other Reserves,Budget sources and other funds,MINORITY INTERESTS,...,Fixed Asset Turnover,Days Sales Outstanding,Days Inventory Outstanding,Days Payable Outstanding,Cash Cycle,Inventory Turnover,(ST+LT borrowings)/Equity,Debt/Equity,Fixed Asset-To-Equity,Owners' Equity/Charter Capital
0,MBB,2024,1,0,0,238749000000,0,0,0,4066862000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.887256,0.049770,1.916773
1,MBB,2023,4,0,0,240294000000,0,0,0,3973110000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.770885,0.050193,1.829197
2,MBB,2023,3,0,0,231594000000,0,0,0,3914443000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.891172,0.052601,1.735607
3,MBB,2023,2,0,0,233122000000,0,0,0,3895569000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.367327,0.057262,1.627913
4,MBB,2023,1,0,0,234649000000,0,0,0,3845711000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.974684,0.059039,1.603294


### MSB

In [25]:
#create period column
msb_bs['period'] = msb_bs.apply(lambda row: create_period(row['yearReport'], row['lengthReport']), axis=1)
msb_ratio.columns = msb_ratio.columns.get_level_values(1)
# Remove duplicated columns, keeping only the first occurrence
msb_bs = msb_bs.loc[:, ~msb_bs.columns.duplicated(keep='first')]
msb_cf = msb_cf.loc[:, ~msb_cf.columns.duplicated(keep='first')]
msb_ic = msb_ic.loc[:, ~msb_ic.columns.duplicated(keep='first')]
msb = pd.concat([msb_bs,msb_ic.iloc[:,3:],msb_cf.iloc[:,3:],msb_ratio.iloc[:,3:]], axis = 1)
msb = msb.replace(np.nan, 0.0)
msb.head()

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16352\1252875303.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  msb = msb.replace(np.nan, 0.0)


,ticker,yearReport,lengthReport,Net Inventories,Other current assets,Investment in properties,Other non-current assets,Other Reserves,Budget sources and other funds,MINORITY INTERESTS,...,Fixed Asset Turnover,Days Sales Outstanding,Days Inventory Outstanding,Days Payable Outstanding,Cash Cycle,Inventory Turnover,(ST+LT borrowings)/Equity,Debt/Equity,Fixed Asset-To-Equity,Owners' Equity/Charter Capital
0,MSB,2024,1,0.0,0.0,0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.629886,0.011743,1.615256
1,MSB,2023,4,0.0,0.0,0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.531025,0.012868,1.564911
2,MSB,2023,3,0.0,0.0,0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.154066,0.010718,1.528381
3,MSB,2023,2,0.0,0.0,0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.056660,0.011795,1.475897
4,MSB,2023,1,0.0,0.0,0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.410682,0.013037,1.399846


### OCB

In [26]:
#create period column
ocb_bs['period'] = ocb_bs.apply(lambda row: create_period(row['yearReport'], row['lengthReport']), axis=1)
ocb_ratio.columns = ocb_ratio.columns.get_level_values(1)
# Remove duplicated columns, keeping only the first occurrence
ocb_bs = ocb_bs.loc[:, ~ocb_bs.columns.duplicated(keep='first')]
ocb_cf = ocb_cf.loc[:, ~ocb_cf.columns.duplicated(keep='first')]
ocb_ic = ocb_ic.loc[:, ~ocb_ic.columns.duplicated(keep='first')]
ocb = pd.concat([ocb_bs,ocb_ic.iloc[:,3:],ocb_cf.iloc[:,3:],ocb_ratio.iloc[:,3:]], axis = 1)
ocb = ocb.replace(np.nan, 0.0)
ocb.head()

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16352\3261006717.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ocb = ocb.replace(np.nan, 0.0)


,ticker,yearReport,lengthReport,Net Inventories,Other current assets,Investment in properties,Other non-current assets,Other Reserves,Budget sources and other funds,MINORITY INTERESTS,...,Fixed Asset Turnover,Days Sales Outstanding,Days Inventory Outstanding,Days Payable Outstanding,Cash Cycle,Inventory Turnover,(ST+LT borrowings)/Equity,Debt/Equity,Fixed Asset-To-Equity,Owners' Equity/Charter Capital
0,OCB,2024,1,0.0,0.0,0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.034419,0.019710,1.435430
1,OCB,2023,4,0.0,0.0,0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.414521,0.018856,1.431310
2,OCB,2023,3,0.0,0.0,0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.646099,0.019199,1.379602
3,OCB,2023,2,0.0,0.0,0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.747479,0.020775,1.327233
4,OCB,2023,1,0.0,0.0,0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.642669,0.022160,1.268064


### STB

In [27]:
#create period column
stb_bs['period'] = stb_bs.apply(lambda row: create_period(row['yearReport'], row['lengthReport']), axis=1)
stb_ratio.columns = stb_ratio.columns.get_level_values(1)
# Remove duplicated columns, keeping only the first occurrence
stb_bs = stb_bs.loc[:, ~stb_bs.columns.duplicated(keep='first')]
stb_cf = stb_cf.loc[:, ~stb_cf.columns.duplicated(keep='first')]
stb_ic = stb_ic.loc[:, ~stb_ic.columns.duplicated(keep='first')]
stb = pd.concat([stb_bs,stb_ic.iloc[:,3:],stb_cf.iloc[:,3:],stb_ratio.iloc[:,3:]], axis = 1)
stb = stb.replace(np.nan, 0.0)
stb.head()

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16352\2514463148.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  stb = stb.replace(np.nan, 0.0)


,ticker,yearReport,lengthReport,Net Inventories,Other current assets,Investment in properties,Other non-current assets,Other Reserves,Budget sources and other funds,MINORITY INTERESTS,...,Fixed Asset Turnover,Days Sales Outstanding,Days Inventory Outstanding,Days Payable Outstanding,Cash Cycle,Inventory Turnover,(ST+LT borrowings)/Equity,Debt/Equity,Fixed Asset-To-Equity,Owners' Equity/Charter Capital
0,STB,2024,1,0.0,0.0,0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.476081,0.153960,2.541301
1,STB,2023,4,0.0,0.0,0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.745958,0.158951,2.425922
2,STB,2023,3,0.0,0.0,0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.900217,0.165216,2.318567
3,STB,2023,2,0.0,0.0,0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.887121,0.170491,2.216882
4,STB,2023,1,0.0,0.0,0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.712191,0.180645,2.151361


### TCB

In [28]:
#create period column
tcb_bs['period'] = tcb_bs.apply(lambda row: create_period(row['yearReport'], row['lengthReport']), axis=1)
tcb_ratio.columns = tcb_ratio.columns.get_level_values(1)
# Remove duplicated columns, keeping only the first occurrence
tcb_bs = tcb_bs.loc[:, ~tcb_bs.columns.duplicated(keep='first')]
tcb_cf = tcb_cf.loc[:, ~tcb_cf.columns.duplicated(keep='first')]
tcb_ic = tcb_ic.loc[:, ~tcb_ic.columns.duplicated(keep='first')]
tcb = pd.concat([tcb_bs,tcb_ic.iloc[:,3:],tcb_cf.iloc[:,3:],tcb_ratio.iloc[:,3:]], axis = 1)
tcb = tcb.replace(np.nan, 0.0)
tcb.head()

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16352\3051435442.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  tcb = tcb.replace(np.nan, 0.0)


,ticker,yearReport,lengthReport,Net Inventories,Other current assets,Investment in properties,Other non-current assets,Other Reserves,Budget sources and other funds,MINORITY INTERESTS,...,Fixed Asset Turnover,Days Sales Outstanding,Days Inventory Outstanding,Days Payable Outstanding,Cash Cycle,Inventory Turnover,(ST+LT borrowings)/Equity,Debt/Equity,Fixed Asset-To-Equity,Owners' Equity/Charter Capital
0,TCB,2024,2,0.0,0.0,0,0.0,0.0,0.0,1381928000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.555726,0.069482,1.966660
1,TCB,2024,1,0.0,0.0,0,0.0,0.0,0.0,1372839000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.423703,0.067154,1.957023
2,TCB,2023,4,0.0,0.0,0,0.0,0.0,0.0,1316833000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.454243,0.067559,1.868384
3,TCB,2023,3,0.0,0.0,0,0.0,0.0,0.0,1274127000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.145520,0.067432,1.804534
4,TCB,2023,2,0.0,0.0,0,0.0,0.0,0.0,1218365000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.981059,0.071387,1.738319


### TPB

In [29]:
#create period column
tpb_bs['period'] = tpb_bs.apply(lambda row: create_period(row['yearReport'], row['lengthReport']), axis=1)
tpb_ratio.columns = tpb_ratio.columns.get_level_values(1)
# Remove duplicated columns, keeping only the first occurrence
tpb_bs = tpb_bs.loc[:, ~tpb_bs.columns.duplicated(keep='first')]
tpb_cf = tpb_cf.loc[:, ~tpb_cf.columns.duplicated(keep='first')]
tpb_ic = tpb_ic.loc[:, ~tpb_ic.columns.duplicated(keep='first')]
tpb = pd.concat([tpb_bs,tpb_ic.iloc[:,3:],tpb_cf.iloc[:,3:],tpb_ratio.iloc[:,3:]], axis = 1)
tpb = tpb.replace(np.nan, 0.0)
tpb.head()

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16352\2615160444.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  tpb = tpb.replace(np.nan, 0.0)


,ticker,yearReport,lengthReport,Net Inventories,Other current assets,Investment in properties,Other non-current assets,Other Reserves,Budget sources and other funds,MINORITY INTERESTS,...,Fixed Asset Turnover,Days Sales Outstanding,Days Inventory Outstanding,Days Payable Outstanding,Cash Cycle,Inventory Turnover,(ST+LT borrowings)/Equity,Debt/Equity,Fixed Asset-To-Equity,Owners' Equity/Charter Capital
0,TPB,2024,1,0.0,0.0,0,0.0,0.0,0.0,154000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.408548,0.029210,1.552946
1,TPB,2023,4,0.0,0.0,0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.891943,0.032340,1.487200
2,TPB,2023,3,0.0,0.0,0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.634981,0.034049,1.470904
3,TPB,2023,2,0.0,0.0,0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.082666,0.034462,1.407406
4,TPB,2023,1,0.0,0.0,0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.204956,0.033716,1.528968


### VCB

In [30]:
#create period column
vcb_bs['period'] = vcb_bs.apply(lambda row: create_period(row['yearReport'], row['lengthReport']), axis=1)
vcb_ratio.columns = vcb_ratio.columns.get_level_values(1)
# Remove duplicated columns, keeping only the first occurrence
vcb_bs = vcb_bs.loc[:, ~vcb_bs.columns.duplicated(keep='first')]
vcb_cf = vcb_cf.loc[:, ~vcb_cf.columns.duplicated(keep='first')]
vcb_ic = vcb_ic.loc[:, ~vcb_ic.columns.duplicated(keep='first')]
vcb = pd.concat([vcb_bs,vcb_ic.iloc[:,3:],vcb_cf.iloc[:,3:],vcb_ratio.iloc[:,3:]], axis = 1)
vcb = vcb.replace(np.nan, 0.0)
vcb.head()

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16352\292421366.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  vcb = vcb.replace(np.nan, 0.0)


,ticker,yearReport,lengthReport,Net Inventories,Other current assets,Investment in properties,Other non-current assets,Other Reserves,Budget sources and other funds,MINORITY INTERESTS,...,Fixed Asset Turnover,Days Sales Outstanding,Days Inventory Outstanding,Days Payable Outstanding,Cash Cycle,Inventory Turnover,(ST+LT borrowings)/Equity,Debt/Equity,Fixed Asset-To-Equity,Owners' Equity/Charter Capital
0,VCB,2024,1,0.0,0.0,0,0.0,0.0,0.0,100556000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.216413,0.043710,3.106303
1,VCB,2023,4,0.0,0.0,0,0.0,0.0,0.0,93990000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.148315,0.045734,3.015577
2,VCB,2023,3,0.0,0.0,0,0.0,0.0,0.0,104660000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.872063,0.046732,2.849184
3,VCB,2023,2,0.0,0.0,0,0.0,0.0,0.0,98316000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.212240,0.049990,2.719604
4,VCB,2023,1,0.0,0.0,0,0.0,0.0,0.0,93744000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.764083,0.053450,2.588227


### VIB

In [31]:
#create period column
vib_bs['period'] = vib_bs.apply(lambda row: create_period(row['yearReport'], row['lengthReport']), axis=1)
vib_ratio.columns = vib_ratio.columns.get_level_values(1)
# Remove duplicated columns, keeping only the first occurrence
vib_bs = vib_bs.loc[:, ~vib_bs.columns.duplicated(keep='first')]
vib_cf = vib_cf.loc[:, ~vib_cf.columns.duplicated(keep='first')]
vib_ic = vib_ic.loc[:, ~vib_ic.columns.duplicated(keep='first')]
vib = pd.concat([vib_bs,vib_ic.iloc[:,3:],vib_cf.iloc[:,3:],vib_ratio.iloc[:,3:]], axis = 1)
vib = vib.replace(np.nan, 0.0)
vib.head()

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16352\3843515908.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  vib = vib.replace(np.nan, 0.0)


,ticker,yearReport,lengthReport,Net Inventories,Other current assets,Investment in properties,Other non-current assets,Other Reserves,Budget sources and other funds,MINORITY INTERESTS,...,Fixed Asset Turnover,Days Sales Outstanding,Days Inventory Outstanding,Days Payable Outstanding,Cash Cycle,Inventory Turnover,(ST+LT borrowings)/Equity,Debt/Equity,Fixed Asset-To-Equity,Owners' Equity/Charter Capital
0,VIB,2024,1,0.0,0.0,2466000000,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.773917,0.020649,1.514333
1,VIB,2023,4,0.0,0.0,2501000000,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.803528,0.019931,1.495539
2,VIB,2023,3,0.0,0.0,2537000000,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.666529,0.020219,1.420674
3,VIB,2023,2,0.0,0.0,2572000000,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.166752,0.020547,1.336713
4,VIB,2023,1,0.0,0.0,2607000000,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.968456,0.020753,1.283914


### VPB

In [32]:
#create period column
vpb_bs['period'] = vpb_bs.apply(lambda row: create_period(row['yearReport'], row['lengthReport']), axis=1)
vpb_ratio.columns = vpb_ratio.columns.get_level_values(1)
# Remove duplicated columns, keeping only the first occurrence
vpb_bs = vpb_bs.loc[:, ~vpb_bs.columns.duplicated(keep='first')]
vpb_cf = vpb_cf.loc[:, ~vpb_cf.columns.duplicated(keep='first')]
vpb_ic = vpb_ic.loc[:, ~vpb_ic.columns.duplicated(keep='first')]
vpb = pd.concat([vpb_bs,vpb_ic.iloc[:,3:],vpb_cf.iloc[:,3:],vpb_ratio.iloc[:,3:]], axis = 1)
vpb = vpb.replace(np.nan, 0.0)
vpb.head()

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16352\3371692566.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  vpb = vpb.replace(np.nan, 0.0)


,ticker,yearReport,lengthReport,Net Inventories,Other current assets,Investment in properties,Other non-current assets,Other Reserves,Budget sources and other funds,MINORITY INTERESTS,...,Fixed Asset Turnover,Days Sales Outstanding,Days Inventory Outstanding,Days Payable Outstanding,Cash Cycle,Inventory Turnover,(ST+LT borrowings)/Equity,Debt/Equity,Fixed Asset-To-Equity,Owners' Equity/Charter Capital
0,VPB,2024,1,0.0,0.0,0,0.0,0.0,0.0,4739680000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.753336,0.013407,1.801598
1,VPB,2023,4,0.0,0.0,0,0.0,0.0,0.0,5232477000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.848305,0.013747,1.763901
2,VPB,2023,3,0.0,0.0,0,0.0,0.0,0.0,5151397000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.093388,0.017264,1.386345
3,VPB,2023,2,0.0,0.0,0,0.0,0.0,0.0,5148465000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.877421,0.016354,1.355746
4,VPB,2023,1,0.0,0.0,0,0.0,0.0,0.0,5751027000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.447564,0.016962,1.324662


In [35]:
financial_statement = pd.concat([acb,bid,ctg,eib,hdb,lpb,mbb,msb,ocb,stb,tcb,tpb,vcb,vib,vpb], ignore_index = True)
financial_statement['Date'] = financial_statement.apply(lambda row: create_period(row['yearReport'], row['lengthReport']), axis=1)
financial_statement

,ticker,yearReport,lengthReport,Net Inventories,Other current assets,Investment in properties,Other non-current assets,Other Reserves,Budget sources and other funds,MINORITY INTERESTS,...,Days Sales Outstanding,Days Inventory Outstanding,Days Payable Outstanding,Cash Cycle,Inventory Turnover,(ST+LT borrowings)/Equity,Debt/Equity,Fixed Asset-To-Equity,Owners' Equity/Charter Capital,Date
0,ACB,2024,2,0.0,0.0,177005000000,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,9.290830,0.062380,1.674466,20240630
1,ACB,2024,1,0.0,0.0,177005000000,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,8.724440,0.062450,1.674423,20240331
2,ACB,2023,4,0.0,0.0,177005000000,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,9.130151,0.067120,1.588569,20231231
3,ACB,2023,3,0.0,0.0,177005000000,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,8.704559,0.066554,1.496091,20230930
4,ACB,2023,2,0.0,0.0,177005000000,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,9.035334,0.068278,1.407477,20230630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648,VPB,2014,1,0.0,0.0,28712000000,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,14.694205,0.051683,0.099401,20140331
649,VPB,2013,4,0.0,0.0,28891000000,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,14.694205,0.054165,0.097388,20131231
650,VPB,2013,3,0.0,0.0,29069000000,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,15.826128,0.054735,0.089573,20130930
651,VPB,2013,2,0.0,0.0,29247000000,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,14.455177,0.058428,0.085849,20130630


# History price

In [2]:
#ACB
stock = Vnstock().stock(symbol='ACB', source='VCI')
acb_hc = stock.quote.history(start='2013-01-01', end='2024-07-27', interval = '1D')
acb_hc['Ma CK'] = 'ACB'
acb_hc.head()

2024-07-28 16:52:20,086 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


,time,open,high,low,close,volume,Ma CK
0,2013-01-02 07:00:00,2.73,2.91,2.73,2.90,2578600,ACB
1,2013-01-03 07:00:00,2.93,2.98,2.80,2.85,1967200,ACB
2,2013-01-04 07:00:00,2.83,2.88,2.80,2.88,1170100,ACB
3,2013-01-07 07:00:00,2.85,2.93,2.85,2.88,1355700,ACB
4,2013-01-08 07:00:00,2.86,2.93,2.83,2.90,2180000,ACB


In [3]:
#BID
stock = Vnstock().stock(symbol='BID', source='VCI')
bid_hc = stock.quote.history(start='2013-01-01', end='2024-07-27', interval = '1D')
bid_hc['Ma CK'] = 'BID'
bid_hc.head()

2024-07-28 16:52:22,278 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


,time,open,high,low,close,volume,Ma CK
0,2014-01-24 07:00:00,9.96,10.22,9.70,9.85,8417060,BID
1,2014-01-27 07:00:00,9.80,9.85,9.49,9.65,3240910,BID
2,2014-02-06 07:00:00,9.54,9.59,9.23,9.23,959590,BID
3,2014-02-07 07:00:00,9.12,9.17,8.65,8.70,2987600,BID
4,2014-02-10 07:00:00,8.65,9.02,8.49,8.91,4052090,BID


In [4]:
#CTG
stock = Vnstock().stock(symbol='CTG', source='VCI')
ctg_hc = stock.quote.history(start='2013-01-01', end='2024-07-27', interval = '1D')
ctg_hc['Ma CK'] = 'CTG'
ctg_hc.head()

2024-07-28 16:52:23,203 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


,time,open,high,low,close,volume,Ma CK
0,2013-01-02 07:00:00,9.92,10.07,9.73,10.07,885890,CTG
1,2013-01-03 07:00:00,10.02,10.02,9.78,9.97,935770,CTG
2,2013-01-04 07:00:00,9.92,10.16,9.83,10.02,627400,CTG
3,2013-01-07 07:00:00,10.26,10.26,9.97,10.07,1796680,CTG
4,2013-01-08 07:00:00,10.02,10.16,9.97,10.16,756690,CTG


In [5]:
#EIB
stock = Vnstock().stock(symbol='EIB', source='VCI')
eib_hc = stock.quote.history(start='2013-01-01', end='2024-07-27', interval = '1D')
eib_hc['Ma CK'] = 'EIB'
eib_hc.head()

2024-07-28 16:52:24,555 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


,time,open,high,low,close,volume,Ma CK
0,2013-01-02 07:00:00,9.86,10.11,9.86,10.05,739550,EIB
1,2013-01-03 07:00:00,10.05,10.05,9.86,9.92,671980,EIB
2,2013-01-04 07:00:00,9.92,10.05,9.86,9.99,348480,EIB
3,2013-01-07 07:00:00,10.05,10.05,9.92,9.99,453580,EIB
4,2013-01-08 07:00:00,9.92,10.24,9.92,10.18,1338620,EIB


In [6]:
#HDB
stock = Vnstock().stock(symbol='HDB', source='VCI')
hdb_hc = stock.quote.history(start='2013-01-01', end='2024-07-27', interval = '1D')
hdb_hc['Ma CK'] = 'HDB'
hdb_hc.head()

2024-07-28 16:52:26,438 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


,time,open,high,low,close,volume,Ma CK
0,2018-01-05 07:00:00,11.56,11.74,10.82,11.74,32205910,HDB
1,2018-01-08 07:00:00,11.86,11.86,11.74,11.77,3321420,HDB
2,2018-01-09 07:00:00,11.77,11.80,11.53,11.79,4537340,HDB
3,2018-01-10 07:00:00,11.62,11.95,11.59,11.95,8726070,HDB
4,2018-01-11 07:00:00,11.98,12.60,11.95,12.60,9874500,HDB


In [7]:
#LPB
stock = Vnstock().stock(symbol='LPB', source='VCI')
lpb_hc = stock.quote.history(start='2013-01-01', end='2024-07-27', interval = '1D')
lpb_hc['Ma CK'] = 'LPB'
lpb_hc.head()

2024-07-28 16:52:27,729 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


,time,open,high,low,close,volume,Ma CK
0,2017-10-05 07:00:00,5.61,5.99,5.11,5.38,7293500,LPB
1,2017-10-06 07:00:00,5.38,5.38,5.15,5.23,1196846,LPB
2,2017-10-09 07:00:00,5.23,5.30,5.19,5.27,1018740,LPB
3,2017-10-10 07:00:00,5.27,5.30,5.15,5.19,1369264,LPB
4,2017-10-11 07:00:00,5.23,5.23,4.89,4.96,1929895,LPB


In [8]:
#MBB
stock = Vnstock().stock(symbol='MBB', source='VCI')
mbb_hc = stock.quote.history(start='2013-01-01', end='2024-07-27', interval = '1D')
mbb_hc['Ma CK'] = 'MBB'
mbb_hc.head()

2024-07-28 16:52:28,915 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


,time,open,high,low,close,volume,Ma CK
0,2013-01-02 07:00:00,2.88,2.99,2.88,2.99,3467810,MBB
1,2013-01-03 07:00:00,3.01,3.01,2.90,2.94,2020300,MBB
2,2013-01-04 07:00:00,2.94,2.99,2.90,2.97,1401390,MBB
3,2013-01-07 07:00:00,2.97,3.01,2.94,2.99,1625590,MBB
4,2013-01-08 07:00:00,2.97,3.13,2.97,3.10,5318350,MBB


In [9]:
#MSB
stock = Vnstock().stock(symbol='MSB', source='VCI')
msb_hc = stock.quote.history(start='2013-01-01', end='2024-07-27', interval = '1D')
msb_hc['Ma CK'] = 'MSB'
msb_hc.head()

2024-07-28 16:52:29,915 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


,time,open,high,low,close,volume,Ma CK
0,2020-12-23 07:00:00,10.65,10.65,10.03,10.06,30431340,MSB
1,2020-12-24 07:00:00,10.35,10.35,10.00,10.24,7683250,MSB
2,2020-12-25 07:00:00,10.24,10.95,10.24,10.95,9311200,MSB
3,2020-12-28 07:00:00,11.24,11.24,10.95,11.09,8600300,MSB
4,2020-12-29 07:00:00,11.15,11.24,10.95,11.24,9680050,MSB


In [10]:
#OCB
stock = Vnstock().stock(symbol='OCB', source='VCI')
ocb_hc = stock.quote.history(start='2013-01-01', end='2024-07-27', interval = '1D')
ocb_hc['Ma CK'] = 'OCB'
ocb_hc.head()

2024-07-28 16:52:30,815 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


,time,open,high,low,close,volume,Ma CK
0,2021-01-28 07:00:00,9.79,11.15,9.79,9.79,7273800,OCB
1,2021-01-29 07:00:00,10.45,10.45,9.92,10.45,6203000,OCB
2,2021-02-01 07:00:00,11.09,11.09,10.45,10.51,6397000,OCB
3,2021-02-02 07:00:00,10.67,10.85,10.45,10.85,4126000,OCB
4,2021-02-03 07:00:00,10.99,11.41,10.88,11.41,5828400,OCB


In [11]:
#STB
stock = Vnstock().stock(symbol='STB', source='VCI')
stb_hc = stock.quote.history(start='2013-01-01', end='2024-07-27', interval = '1D')
stb_hc['Ma CK'] = 'STB'
stb_hc.head()

2024-07-28 16:52:32,144 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


,time,open,high,low,close,volume,Ma CK
0,2013-01-02 07:00:00,13.52,13.66,13.32,13.66,252100,STB
1,2013-01-03 07:00:00,13.66,13.93,13.52,13.93,260820,STB
2,2013-01-04 07:00:00,13.93,14.14,13.66,14.07,271370,STB
3,2013-01-07 07:00:00,14.07,14.61,13.73,14.61,825340,STB
4,2013-01-08 07:00:00,14.61,14.88,14.34,14.88,531040,STB


In [12]:
#TCB
stock = Vnstock().stock(symbol='TCB', source='VCI')
tcb_hc = stock.quote.history(start='2013-01-01', end='2024-07-27', interval = '1D')
tcb_hc['Ma CK'] = 'TCB'
tcb_hc.head()

2024-07-28 16:52:33,675 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


,time,open,high,low,close,volume,Ma CK
0,2018-06-04 07:00:00,16.55,16.97,16.55,16.55,2811840,TCB
1,2018-06-05 07:00:00,16.02,17.13,15.52,15.52,1357500,TCB
2,2018-06-06 07:00:00,15.35,15.52,14.71,14.87,1605480,TCB
3,2018-06-07 07:00:00,15.27,15.90,15.05,15.90,1376540,TCB
4,2018-06-08 07:00:00,16.32,17.00,16.08,17.00,2008500,TCB


In [13]:
#TPB
stock = Vnstock().stock(symbol='TPB', source='VCI')
tpb_hc = stock.quote.history(start='2013-01-01', end='2024-07-27', interval = '1D')
tpb_hc['Ma CK'] = 'TPB'
tpb_hc.head()

2024-07-28 16:52:34,911 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


,time,open,high,low,close,volume,Ma CK
0,2018-04-19 07:00:00,10.43,10.43,9.48,9.67,7295110,TPB
1,2018-04-20 07:00:00,9.78,9.78,9.51,9.69,1970610,TPB
2,2018-04-23 07:00:00,9.69,9.72,9.03,9.18,918490,TPB
3,2018-04-24 07:00:00,9.03,9.24,8.56,9.18,1084620,TPB
4,2018-04-26 07:00:00,9.15,9.21,8.79,8.85,618040,TPB


In [14]:
#VCB
stock = Vnstock().stock(symbol='VCB', source='VCI')
vcb_hc = stock.quote.history(start='2013-01-01', end='2024-07-27', interval = '1D')
vcb_hc['Ma CK'] = 'VCB'
vcb_hc.head()

2024-07-28 16:52:35,827 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


,time,open,high,low,close,volume,Ma CK
0,2013-01-02 07:00:00,9.69,9.90,9.69,9.90,168190,VCB
1,2013-01-03 07:00:00,9.94,10.15,9.62,10.15,500650,VCB
2,2013-01-04 07:00:00,9.97,10.51,9.90,10.40,501540,VCB
3,2013-01-07 07:00:00,9.97,10.69,9.97,10.69,1312740,VCB
4,2013-01-08 07:00:00,10.90,11.22,10.72,11.22,1035220,VCB


In [15]:
#VIB
stock = Vnstock().stock(symbol='VIB', source='VCI')
vib_hc = stock.quote.history(start='2013-01-01', end='2024-07-27', interval = '1D')
vib_hc['Ma CK'] = 'VIB'
vib_hc.head()

2024-07-28 16:52:37,108 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


,time,open,high,low,close,volume,Ma CK
0,2017-01-09 07:00:00,4.30,4.30,3.07,3.34,346000,VIB
1,2017-01-10 07:00:00,3.45,3.45,3.34,3.38,234947,VIB
2,2017-01-11 07:00:00,3.38,3.38,3.31,3.34,173442,VIB
3,2017-01-12 07:00:00,3.25,3.25,3.04,3.04,52349,VIB
4,2017-01-13 07:00:00,2.95,2.95,2.73,2.75,70740,VIB


In [16]:
#VPB
stock = Vnstock().stock(symbol='VPB', source='VCI')
vpb_hc = stock.quote.history(start='2013-01-01', end='2024-07-27', interval = '1D')
vpb_hc['Ma CK'] = 'VPB'
vpb_hc.head()

2024-07-28 16:52:38,286 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


,time,open,high,low,close,volume,Ma CK
0,2017-08-17 07:00:00,8.08,8.08,6.84,8.08,58299310,VPB
1,2017-08-18 07:00:00,7.87,7.98,7.71,7.71,1092840,VPB
2,2017-08-21 07:00:00,7.42,7.62,7.36,7.43,798670,VPB
3,2017-08-22 07:00:00,7.21,7.46,7.21,7.46,2295930,VPB
4,2017-08-23 07:00:00,7.46,7.62,7.41,7.50,520790,VPB


In [17]:
history_price = pd.concat([acb_hc,bid_hc,ctg_hc,eib_hc,hdb_hc,lpb_hc,mbb_hc,msb_hc,ocb_hc,stb_hc,tcb_hc,tpb_hc,vcb_hc,vib_hc,vpb_hc], ignore_index = True)
history_price

,time,open,high,low,close,volume,Ma CK
0,2013-01-02 07:00:00,2.73,2.91,2.73,2.90,2578600,ACB
1,2013-01-03 07:00:00,2.93,2.98,2.80,2.85,1967200,ACB
2,2013-01-04 07:00:00,2.83,2.88,2.80,2.88,1170100,ACB
3,2013-01-07 07:00:00,2.85,2.93,2.85,2.88,1355700,ACB
4,2013-01-08 07:00:00,2.86,2.93,2.83,2.90,2180000,ACB
...,...,...,...,...,...,...,...
31742,2024-07-22 07:00:00,18.80,19.05,18.65,18.80,15851459,VPB
31743,2024-07-23 07:00:00,18.85,18.90,18.20,18.40,17896447,VPB
31744,2024-07-24 07:00:00,18.35,18.60,18.20,18.50,13371089,VPB
31745,2024-07-25 07:00:00,18.40,18.50,18.30,18.35,10461655,VPB


In [18]:
financial_statement.to_csv(r'C:\Users\LENOVO\Downloads\financial_statment.csv')
history_price.to_csv(r'C:\Users\LENOVO\Downloads\history_price.csv')